&emsp;&emsp;初始化一个深度网络时,为了缓解梯度消失或爆炸问题,应尽可能保持每个神经元的输入和输出的方差一致,
根据神经元的连接数量来自适应地调整初始化分布的方差,这类方法称为方差缩放(Variance Scaling).

### Xavier初始化

&emsp;&emsp;假设在一个神经网络中,第$l$层的一个神经元$a^{l}_j$,接受前一层的$N_{l-1}$个神经元的输出$\mathbf{o}^{l-1}$

$$ a^{l}_j = f(\sum_{i=1}^{N_{l-1}} w^l_{j,i} o^{l-1}_i) $$

其中$f(*)$为激活函数,$w^l_{j,i}$为权重参数,$N_{l-1}$是第$l-1$层神经元个数.为简化起见.这里假设激活函数为恒等函数,即$f(x)=x$.

&emsp;&emsp;<font color='red'>假设</font>$\mathbf{w}_{j,}^l$均值为0,并且与$\boldsymbol{o}^{l-1}$相互独立,则$a^{l}$的均值为

$$ E\left[a^{l}_j\right]=E\left[\sum_{i=1}^{N_{l-1}} w_{j,i}^{l} o_{i}^{(l-1)}\right]=\sum_{i=1}^{N_{l-1}} E\left[w_{j,i}^{(l)}\right] \mathbb{E}\left[o_{i}^{(l-1)}\right]=0 $$

$a^{l}_j $的方差为

\begin{align}
var(a^l_j) &= var \left(\sum_{i=1}^{N_{l-1}} w^l_{j,i} o^{l-1}_i \right) \\
		 &=N_{l-1} var(w^l_{j,i}) var(o^{l-1}_i)
\end{align}

也就是说,输入信号的方差经过该神经元后被放大或缩小了$N_{l-1} var(w^l_{j,i})$倍,为了使得在经过多层网络后,信号不被过分放大或过分减弱,应尽可能保持每个神经元的输入和输出的方差一致.这样$N_{l-1} var(w^l_{j,i})$设为1 比较合理,即

$$ var(w^l_{j,i}) = \frac{1}{N_{l-1}} $$

&emsp;&emsp;同理,在反向传播过程中,由

$$ \boldsymbol{\delta}^{l-1} = (W^{l})^T \boldsymbol{\delta}^{l} $$

$$ \delta^{l-1}_i = \sum_{j=1}^{N_l} w_{j, i}^l \delta_j^l $$

<font color='red'>假设</font>$\mathbf{w}^l_{,i}$均值为0,并且与$\boldsymbol{\delta}^{l}$相互独立,为了误差信号也不被放大或缩小,需要将$w^{l}_{j,i}$的方差保持为

$$ var(w^l_{j,i}) = \frac{1}{N_{l}} $$

作为折中,同时考虑信号在前向和反向传播中都不被放大或缩小,可以设置

$$ var(w^l_{j,i}) = \frac{2}{N_{l} + M_{l-1}} $$

在计算出参数的理想方差后,可以通过高斯分布或均匀分布来随机初始化参数.若采用高斯分布来随机初始化参数,
连接权重$w^l_{j,i}$可以按$N(0, \frac{2}{N_{l} + M_{l-1}})$的高斯分布进行初始化.若采用区间为$[−r, r]$的均分分布来初始化$w^l_{j,i}$,
则$r$的取值为$\sqrt{\frac{6}{N_{l} + M_{l-1}}}$.种根据每层的神经元数量来自动计算初始化参数方差的方法称为Xavier 初始化.

&emsp;&emsp;虽然在Xavier初始化中我们假设激活函数为恒等函数,但是Xavier初始化也适用于Sigmoid函数和Tanh函数.这是因为神经元的参数和输入的绝对值通常比较小,
处于激活函数的线性区间,在实际应用中,使用Sigmoid函数或Tanh函数的神经层通常将方差$\frac{2}{N_{l} + M_{l-1}}$乘以一个缩放因子$\rho$.(可参考nn.init.calculate_gain函数)

***

&emsp;&emsp;在一个给定的区间$[−r, r]$ 内采用均匀分布来初始化参数.假设随机变量$X$在区间$[a, b] $内服从均匀分布,则其方差为

$$ var(X) = \frac{(a-b)^2}{12} $$

因此,若使用区间为$[−r, r]$的均分分布来采样,并满足$var(x)=\sigma^2$时,则$r$的取值为

$$ r= \sqrt{3 \sigma^2} $$

***

### Kaiming 初始化

&emsp;&emsp;当第$l$层神经元使用ReLU激活函数时,通常有一半的神经元输出为0,因此其分布的方差也近似为使用恒等函数时的一半.这样,只考虑前向传播时,参数$w^{l}_{j,i}$的理想方差为

$$ var(w^{l}_{j,i}) = \frac{1}{N_{l-1}} $$

只考虑反向传播时,参数$w^{l}_{j,i}$的理想方差为

$$ var(w^{l}_{j,i}) = \frac{1}{N_{l}} $$

因此当使用ReLU激活函数时,若采用高斯分布来初始化参数$w^{l}_{j,i}$,其方差为$\frac{1}{N_{l-1}} \, or \, \frac{1}{N_{l}}$;若采用区间为$[r,r]$的均分分布来初始化参数$w^{l}_{j,i}$,则$r=\sqrt{\frac{3}{N_{l-1}}} \, or \, \sqrt{\frac{3}{N_{l}}} $种初始化方法称为Kaiming 初始化．
